In [1]:
%load_ext autoreload
%autoreload 2

import IPython
IPython.get_ipython().run_line_magic('cd', '..')  # Go up one directory

from dictionary_learning.buffer import AllActivationBuffer
from dictionary_learning.trainers.top_k import TrainerSCAE, AutoEncoderTopK
from dictionary_learning.training import trainSCAE

from datasets import load_dataset
import torch as t
from nnsight import LanguageModel

#
device = "cuda:0" if t.cuda.is_available() else "cpu"
model = LanguageModel("gpt2", device_map=device)
model.eval()


dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

/usr/local/lib/python3.11/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/root


In [2]:
C = 20
expansion = 16
k= 128 # TODO

num_features = model.config.n_embd * expansion

In [3]:
n_layer = model.config.n_layer

submodules = {}
for layer in range(n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")


buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    d_submodule=model.config.n_embd, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower depending on your available memory
    device="cuda",
    out_batch_size = 512,
    refresh_batch_size = 256,
) 

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [4]:
important_features = {f"mlp_{layer}": t.randint(0, num_features, (num_features, C))
                        for layer in range(n_layer)}

# Get submodule names from the submodules dictionary
submodule_names = list(submodules.keys())

# Configure the multi-trainer
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": {name: model.config.n_embd for name in submodule_names},
    "dict_sizes": {name: model.config.n_embd * expansion for name in submodule_names},
    "ks": {name: k for name in submodule_names},
    "device": buffer.device,
    "submodules": submodules,
    "important_features": important_features,
}

trainer_class = trainer_cfg.pop("trainer")
trainer = trainer_class(**trainer_cfg)

In [ ]:
inputs, targets = next(buffer)

In [ ]:
trainer.loss(inputs, targets)

tensor(256201., device='cuda:0', grad_fn=<AddBackward0>)

In [10]:
trainer_cfg = {
    "trainer": TrainerSCAE,
    "activation_dims": {name: model.config.n_embd for name in submodule_names},
    "dict_sizes": {name: model.config.n_embd * expansion for name in submodule_names},
    "ks": {name: k for name in submodule_names},
    "device": buffer.device,
    "submodules": submodules,
    "important_features": important_features,
}

# Run the training
trainer = trainSCAE(
    buffer=buffer,
    trainer_cfg=trainer_cfg,
    steps=30000,
    save_steps=1000,
    save_dir="sae_checkpoints",
    log_steps=100,
    use_wandb=False  # Set to False if you don't want to use wandb
)

#aa9a791c5e40fa7ab2f08d555ff72352c1cecaa2

Loss: 134148.3438:   0%|          | 35/30000 [04:15<60:48:21,  7.31s/it]


KeyboardInterrupt: 

In [11]:
trainer.timings

{'vanilla_encode_mlp_0': [37.3493766784668],
 'vanilla_decode_mlp_0': [0.8058879971504211],
 'vanilla_forward_mlp_0': [49.53209686279297],
 'vanilla_encode_attn_0': [1.213088035583496],
 'vanilla_decode_attn_0': [0.5580800175666809],
 'vanilla_forward_attn_0': [2.357599973678589],
 'vanilla_encode_mlp_1': [1.494271993637085],
 'vanilla_decode_mlp_1': [0.5468159914016724],
 'vanilla_forward_mlp_1': [2.524224042892456],
 'vanilla_encode_attn_1': [1.1706880331039429],
 'vanilla_decode_attn_1': [0.5386239886283875],
 'vanilla_forward_attn_1': [2.1621758937835693],
 'vanilla_encode_mlp_2': [1.1542079448699951],
 'vanilla_decode_mlp_2': [0.5468159914016724],
 'vanilla_forward_mlp_2': [2.1412160396575928],
 'vanilla_encode_attn_2': [1.157312035560608],
 'vanilla_decode_attn_2': [0.536575973033905],
 'vanilla_forward_attn_2': [2.1166720390319824],
 'vanilla_encode_mlp_3': [1.3451839685440063],
 'vanilla_decode_mlp_3': [0.5355520248413086],
 'vanilla_forward_mlp_3': [2.3292479515075684],
 'vani